In [ ]:
import warnings
from sklearn import tree
import numpy as np
import shap
import pandas as pd
import seaborn as sns
from datetime import datetime
from sklearn.base import clone
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support, classification_report, confusion_matrix, SCORERS

In [ ]:
%matplotlib inline
warnings.filterwarnings("ignore")
df = pd.read_csv("../data/accidents.csv")

### Feature Engineering or encoding

#### Feature Selection

In [ ]:
high_missing_val = ['End_Lat', 'End_Lng', 'Number', 'Precipitation(in)']
zero_variance = ['Country', 'Turning_Loop']
high_correlation = ['Wind_Chill(F)']
text_feat = ['Description', 'Weather_Condition']#, 'Street'
other = ['ID']

In [ ]:
def drop_cols(df, col_set):
    df1 = df.copy()
    return df1.drop(columns = col_set)
df1 = drop_cols(df, high_missing_val+zero_variance+high_correlation+text_feat+other)

#### Outlier Detection and Correction

##### Wind Direction

In [ ]:
wind_dir_dict = {'Calm': 'CALM',
                'East': 'E',
                'West': 'W',
                'North': 'N',
                'South': 'S',
                'Variable': 'VAR'}
def replace_vals(x):
    if x in wind_dir_dict:
        return wind_dir_dict[x]
    else:
        return x
df1['Wind_Direction'] = df1['Wind_Direction'].apply(lambda x:replace_vals(x))

##### Wind Speed

In [ ]:
df1['Wind_Speed(mph)'][df1['Wind_Speed(mph)']>231] = 231

### Feature Engineering

In [ ]:
# time of the day
df1['St_HOD'] = df1['Start_Time'].apply(lambda x:int(x.split(' ')[-1].split(':')[0]))
df1['Ed_HOD'] = df1['End_Time'].apply(lambda x:int(x.split(' ')[-1].split(':')[0]))
# day of the week
df1['Accident_DOW'] = pd.to_datetime(df1['Start_Time']).dt.dayofweek
# week of the year
df1['Accident_WOY'] = pd.to_datetime(df1['Start_Time']).dt.week
# weekday/weekend - Although this is a correlated feature to Day of Week. We will let the algorithm choose the better of the two.
df1['Weekday_Weekend'] = 0
df1['Weekday_Weekend'][df1['Accident_DOW'].isin([5,6])] = 1

In [ ]:
# Text Features - Street and Description
# Ave/Avenue, Rd/Road, St/Street, Ln/Lane, Dr/Drive, Way, Pl/Plaza/Square, Blvd/boulevard, Hwy/Highway/Expy/Fwy/Parkway,
# Main, Route
street_dict = {
    0: ['Ave', 'Avenue'],
    1: ['Rd', 'Road'],
    2: ['St', 'Street'],
    3: ['Ln', 'Lane'],
    4: ['Dr', 'Drive'],
    5: ['Pl', 'Plaza', 'Square'],
    6: ['Blvd', 'Boulevard'],
    7: ['Hwy', 'Highway', 'Expy', 'Fwy', 'Parkway'],
    8: ['Main'],
    9: ['Route']
}
def find_street_type(string, street_dict):
    for key in street_dict:
        for tok in street_dict[key]:
            if tok in string.split(' '):
                return key
    return 10
df1['Street'] = df1['Street'].apply(lambda x:find_street_type(x, street_dict))

In [ ]:
### Features: Accident Duration & Time between weather measurement and accident start time
df1['Start_Time'] = pd.to_datetime(df1['Start_Time'])
df1['End_Time'] = pd.to_datetime(df1['End_Time'])
df1['Weather_Timestamp'] = pd.to_datetime(df1['Weather_Timestamp'])
df1['Accident_Duration'] = df1['End_Time'] - df1['Start_Time']
df1['Weather_Impact_Duration'] = df1['Start_Time'] - df1['Weather_Timestamp']
df1 = df1.drop(columns = ['Start_Time', 'End_Time', 'Weather_Timestamp'])

In [ ]:
# Location features
# 'Start_Lat', 'Start_Lng', 'City', 'County', 'State', 'Zipcode', 'Timezone', 'Airport_Code'
# In the first cut, I would select the 'State' feature and remove others mainly because it 
# has medium cardinality and zero nans
df1 = df1[df1.columns.difference(['Start_Lat', 'Start_Lng', 'City', 'County', 'Zipcode', 'Timezone', 'Airport_Code'])]

#### Missing Value Imputation and Label Encoding

In [ ]:
### Categorical features
bool_feat = df1.select_dtypes(include='bool').columns
cat_feat = df1.select_dtypes(include='object').columns
for col in cat_feat:
    df1[col] = df1[col].fillna(df1[col].mode()[0])
    u_val = df1[col].unique()
    val_d = dict([(u_val[k], k) for k in range(len(u_val))])
    df1[col] = df1[col].apply(lambda x:val_d[x])
for col in bool_feat:
    df1[col] = df1[col].apply(lambda x:int(x))

In [ ]:
### Continuous Features with missing values
## TODO: Add distribution to exploratory for confirming mean or median imputation
con_feat = ['Humidity(%)', 'Pressure(in)', 'TMC', 'Temperature(F)', 'Visibility(mi)', 'Weather_Impact_Duration',
           'Wind_Speed(mph)']
for col in con_feat:
    df1[col] = df1[col].fillna(df1[col].mean())

In [ ]:
### Timestamp Features
for col in ['Accident_Duration', 'Weather_Impact_Duration']:
    df1[col] = df1[col].apply(lambda x:x.seconds)

In [ ]:
df1.head().T

### Simple Model

In [ ]:
### Train-Test Split
X = df1[[k for k in df1.columns if k!='Severity']]
y = df1['Severity']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#### Scaling for Linear Models

In [ ]:
X_scaled = X.copy()
for col in X_scaled.columns:
    X_scaled[col] = MinMaxScaler().fit_transform(np.array(X_scaled[col]).reshape(-1, 1))

In [ ]:
X_scaled.head().T

### Cross validation pipeline on training data for model selection

In [ ]:
#### Adjusting for imbalance in 3 common classifiers
cls_wt_dict = dict(max(df1['Severity'].value_counts())/df1['Severity'].value_counts())
f1_macro_score = {}
clf_list = [LogisticRegression(class_weight=cls_wt_dict), DecisionTreeClassifier(class_weight=cls_wt_dict),
            RandomForestClassifier(class_weight=cls_wt_dict)]
for clf in clf_list:
    scores = cross_val_score(clf, X_scaled, y, cv=5, scoring = 'f1_macro')
    f1_macro_score[type(clf).__name__] = (np.mean(scores), np.std(scores))
## Random Forest gives best performance out of the 3 simple classifiers
f1_macro_score

### Feature Importance

In [ ]:
clf.fit(X_train, y_train)
feat_array = clf.feature_importances_
feat_imp = pd.DataFrame([k for k in zip(X_train.columns, feat_array)]).sort_values(1, ascending=False).reset_index().iloc[:16,:][[0,1]]
feat_imp.columns = ['Feature', 'Contribution']
## Top 10 feature contribution
print(feat_imp['Contribution'].sum())
feat_imp

#### Evaluation metric

The evaluation metric was choosen as 'F1 Macro' to make sure that all the classes irrespective of their support get equal weightage in the evaluation.

### Check whether train and test belong from same distribution

In [ ]:
from scipy.stats import ks_2samp
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric_cols = X_train.select_dtypes(include=numerics).columns
def kstest(olddata, newdata, numeric_cols):
    count_list = 0
    same_list, diff_list = [], []
    for col in numeric_cols:
        oldvar = olddata[col]
        newvar = newdata[col]
        (s,p) = ks_2samp(oldvar.values.reshape((1,olddata.shape[0]))[0], newvar.values.reshape((1,newdata.shape[0]))[0])
#         print(col, s, p)
        if p > 0.05:
            same_list.append(col)
#             print("same")
            count_list += 1
        else:
            diff_list.append(col)
    if count_list/len(numeric_cols) > 0.5:
        
        return 'same', same_list, diff_list
    else:
        return 'different', same_list, diff_list

same_diff, same_list, diff_list = kstest(X_train, X_test, numeric_cols)
print('Distribution:',same_diff)
print('Columns with Same Distribution:',same_list)
print('Columns with Different Distribution:',diff_list)

### Run test data through same pipeline

#### Decision Tree

In [ ]:
clf = DecisionTreeClassifier(random_state=0).fit(X_train, y_train)
y_pred = clf.predict(X_test)
precision_recall_fscore_support(y_test, y_pred, average=None, labels=[1,2,3,4])
# (array([0.61814496, 0.822673  , 0.58727317, 0.50919495]),
#  array([0.63507603, 0.81590944, 0.59588685, 0.52772611]),
#  array([0.62649612, 0.81927726, 0.59154866, 0.51829494]),
#  array([  9602, 783560, 329237,  37095]))

#### Random Forest

In [ ]:
clf = RandomForestClassifier(random_state=0).fit(X_train, y_train)
y_pred = clf.predict(X_test)
precision_recall_fscore_support(y_test, y_pred, average=None, labels=[1,2,3,4])
# (array([0.78085678, 0.81562352, 0.68786448, 0.70874232]),
#  array([0.57519267, 0.89299096, 0.56534351, 0.45742014]),
#  array([0.66242879, 0.85255561, 0.62061481, 0.55599974]),
#  array([  9602, 783560, 329237,  37095]))
# with class imbalance correction 
# (array([0.78518212, 0.80759026, 0.69159645, 0.72938191]),
#  array([0.59268902, 0.89965159, 0.54047692, 0.43391293]),
#  array([0.67548961, 0.85113877, 0.60676893, 0.54412386]),
#  array([  9602, 783560, 329237,  37095]))

#### Gradient Boosting

In [ ]:
## High run-time
# clf = GradientBoostingClassifier(random_state=0).fit(X_train, y_train)
# y_pred = clf.predict(X_test)
# precision_recall_fscore_support(y_test, y_pred, average=None, labels=[1,2,3,4])
# (array([0.76084462, 0.80717385, 0.65963367, 0.63435457]),
#  array([0.34524057, 0.87432743, 0.576776  , 0.3164847 ]),
#  array([0.47496239, 0.8394097 , 0.61542849, 0.42228697]),
#  array([  9602, 783560, 329237,  37095]))

#### Support Vector Classifier (1 vs all)

In [ ]:
## Very High run-time (Not recommended)
# clf = OneVsRestClassifier(SVC()).fit(X_train, y_train)
# y_pred = clf.predict(X_test)
# precision_recall_fscore_support(y_test, y_pred, average=None, labels=[1,2,3,4])


#### Ordinal Classification

In [ ]:
y_train = y_train - 1
y_test = y_test - 1

In [ ]:
class OrdinalClassifier():
    
    def __init__(self, clf):
        self.clf = clf
        self.clfs = {}
    
    def fit(self, X, y):
        self.unique_class = np.sort(np.unique(y))
        if self.unique_class.shape[0] > 2:
            for i in range(self.unique_class.shape[0]-1):
                # for each k - 1 ordinal value we fit a binary classification problem
                binary_y = (y > self.unique_class[i]).astype(np.uint8)
                clf = clone(self.clf)
                clf.fit(X, binary_y)
                self.clfs[i] = clf
    
    def predict_proba(self, X):
        clfs_predict = {k:self.clfs[k].predict_proba(X) for k in self.clfs}
        predicted = []
        for i,y in enumerate(self.unique_class):
            if i == 0:
                # V1 = 1 - Pr(y > V1)
                predicted.append(1 - clfs_predict[y][:,1])
            elif y in clfs_predict:
                # Vi = Pr(y > Vi-1) - Pr(y > Vi)
                 predicted.append(clfs_predict[y-1][:,1] - clfs_predict[y][:,1])
            else:
                # Vk = Pr(y > Vk-1)
                predicted.append(clfs_predict[y-1][:,1])
        return np.vstack(predicted).T
    
    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)

In [ ]:
clf = OrdinalClassifier(DecisionTreeClassifier())
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
precision_recall_fscore_support(y_test, y_pred, average=None, labels=[1,2,3,4])

In [ ]:
clf = OrdinalClassifier(RandomForestClassifier(random_state=0))
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
precision_recall_fscore_support(y_test, y_pred, average=None, labels=[1,2,3,4])

#### Power Analysis:
1. Improving train speed
2. Correcting imbalance

In [ ]:
### Undersampling Class 2 and 3 to match class 1 numbers
## Here we are emphasising on improving class 4 Recall.
## it really depends on the business case i.e. if a govt agency wants to ensure avoiding severity 4 traffic jams
sample_size = df1[df1['Severity']==1].shape[0]
df1_1 = df1[df1['Severity']==1]
df1_2 = df1[df1['Severity']==2].sample(sample_size)
df1_3 = df1[df1['Severity']==3].sample(sample_size)
df1_4 = df1[df1['Severity']==4]
df2 = pd.concat([df1_1, df1_2, df1_3, df1_4])
X_train = df2[[k for k in df2.columns if k!='Severity']]
y_train = df2['Severity']
clf = RandomForestClassifier(random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
precision_recall_fscore_support(y_test, y_pred, average=None, labels=[1,2,3,4])

Above we can see that we got 99%+ Recall on Class 4, but pretty poor Precision.
But is this information enough for the govt agency to avoid Severity 4 instances?
The answer is NO.
They need to know their action items which we would explain by SHAP.

### Predictions and Interpretability

In [ ]:
## Picking most confident examples
X_test = X_test.reset_index(drop=True)
prob_mat = clf.predict_proba(X_test)
idx1 = np.argmax([k[0] for k in prob_mat])
idx2 = np.argmax([k[1] for k in prob_mat])
idx3 = np.argmax([k[2] for k in prob_mat])
idx4 = np.argmax([k[3] for k in prob_mat])

In [ ]:
### SHAP Example 1
explainer = shap.TreeExplainer(clf)
choosen_instance = X_test.loc[[idx1]]
shap_values = explainer.shap_values(choosen_instance)
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1], choosen_instance)

In [ ]:
### SHAP Example 2
explainer = shap.TreeExplainer(clf)
choosen_instance = X_test.loc[[idx2]]
shap_values = explainer.shap_values(choosen_instance)
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1], choosen_instance)

In [ ]:
### SHAP Example 3
explainer = shap.TreeExplainer(clf)
choosen_instance = X_test.loc[[idx3]]
shap_values = explainer.shap_values(choosen_instance)
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1], choosen_instance)

In [ ]:
### SHAP Example 4
explainer = shap.TreeExplainer(clf)
choosen_instance = X_test.loc[[idx4]]
shap_values = explainer.shap_values(choosen_instance)
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1], choosen_instance)

In [ ]:
### Class 3 is Severity 4
shap.summary_plot(shap_values, X_train)

In [ ]:
# Interpretibility
### Supply the index of specific test row
train_cols = X_train.columns
row_idx = idx2
max_display  = 1200
data_for_prediction = X_test.loc[row_idx]
data_for_prediction = np.array(data_for_prediction).reshape(1,-1)
# Calculate Shap values
explainer = shap.TreeExplainer(clf)
shap_values = explainer.shap_values(data_for_prediction)

pred_probability = clf.predict_proba(data_for_prediction)
prediction = np.argmax(pred_probability)
print("prediction: ", prediction, pred_probability)

shap_value = shap_values[prediction]

feature_order = np.argsort(np.sum(np.abs(shap_value), axis=0))
feature_order = np.flip(feature_order[-min(max_display, len(feature_order)):], 0)

top_shape_vals = [shap_value[0][i] for i in feature_order]

top_feature_names = [train_cols[i] for i in feature_order]

fig = plt.figure(figsize=(18,15))
y_pos = np.arange(len(top_feature_names))

values = np.flip(top_shape_vals)
features = np.flip(top_feature_names)

plt.barh(y_pos, values, align='center')
plt.yticks(y_pos, fontsize=15)
plt.gca().set_yticklabels(features)
plt.title("Feature Importance")

### Tree Visualization

In [ ]:
important_features = list(feat_imp.Feature)
X_train = df1[important_features]
y_train = df1['Severity']
dt_clf = DecisionTreeClassifier(random_state=0, max_depth = 3).fit(X_train, y_train)
tree.plot_tree(dt_clf)
plt.show()